# Anti-Pattern DADGT Example 2

JPG vs PNG

Case 2, image size is 256 x 256.

In [1]:
from dlcliche.notebook import *
from dlcliche.utils import *
sys.path.append('..')
from dadgt.dadgt_utils import *

deterministic_everything(1234, pytorch=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('GPU：', device)

ORG = Path('/data/mvtec_ad/original/transistor')

ensure_delete('temp')

GPU： cuda:0


In [2]:
class GeoTfm4Dataset(GeoTfmDataset):
    geo_tfms = list(product(
        [None], 
        [None, Image.ROTATE_90, Image.ROTATE_180, Image.ROTATE_270],
        [None]
    ))

SIZE = 256
params = EasyDict()
params.fit = {}
params.fit.validation_split = 0.1
params.fit.lr = 0.003
params.fit.b1 = 0.95
params.fit.b2 = 0.999
params.fit.weight_decay = 0.0
params.fit.batch_size = 64
params.fit.epochs = 50
params.load_size = SIZE + (SIZE // 50)
params.crop_size = SIZE
params.work_folder = 'temp/work'
params.suffix = '.png'
params.project = f'Transistor-GT4-{SIZE}'
params.ds_cls = GeoTfm4Dataset
params.n_class = params.ds_cls.n_tfm()

In [3]:
def make_resized_files(src_files, dest_folder, suffix, load_size):
    dest_folder = Path(dest_folder)
    # ensure_delete(dest_folder)
    ensure_folder(dest_folder)

    files = []
    for file_name in src_files:
        img = Image.open(file_name)
        img = img.convert('RGB').resize((load_size, load_size))
        new_file_name = dest_folder/f'{Path(file_name).stem}{suffix}'
        if suffix == '.jpg':
            img.save(new_file_name) #, quality=30)
        else:
            img.save(new_file_name)
        files.append(new_file_name)
    return files

train_files = make_resized_files((ORG/'train/good').glob('*.png'), dest_folder='temp/train', suffix='.jpg', load_size=params.load_size)

In [4]:
test_good = make_resized_files((ORG/'test/good').glob('*.png'), dest_folder='temp/test/good',
                               suffix='.jpg', load_size=params.load_size)
test_bend = make_resized_files((ORG/'test/bent_lead').glob('*.png'), dest_folder='temp/test/bend',
                               suffix='.png', load_size=params.load_size)
test_cut = make_resized_files((ORG/'test/cut_lead').glob('*.png'), dest_folder='temp/test/cut',
                               suffix='.png', load_size=params.load_size)
test_damaged = make_resized_files((ORG/'test/damaged_case').glob('*.png'), dest_folder='temp/test/damaged',
                               suffix='.png', load_size=params.load_size)

In [5]:
!ls {ORG/'test'}

bent_lead  cut_lead  damaged_case  good  misplaced


In [6]:
train_dataset = params.ds_cls(file_list=train_files, load_size=params.load_size, crop_size=params.crop_size,
                                transform=ImageTransform(), random=True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=params.fit.batch_size, shuffle=True)

# Check
batch_iterator = iter(train_dataloader)
imgs, labels = next(batch_iterator)
print(imgs.size(), labels.size(), len(train_dataset), len(train_dataloader), len(train_dataset.classes()))
del train_dataset, train_dataloader

torch.Size([64, 3, 256, 256]) torch.Size([64]) 852 14 4


In [7]:
#plt_tiled_imshow(imgs[:10], [str(l) for l in labels.detach().cpu().numpy()])

In [ ]:
model = create_model(device, params.n_class, weight_file=None)
learner = TrainingScheme(device, model, params, train_files, params.ds_cls)

trainer = pl.Trainer(max_epochs=params.fit.epochs, gpus=torch.cuda.device_count(), show_progress_bar=False)
trainer.fit(learner);

## Test

- Good: jpg
- Others: png

In [9]:
test_files = test_good + test_bend + test_cut + test_damaged
labels = np.array([0] * len(test_good) + [1] * len(test_bend) + [1] * len(test_cut) + [1] * len(test_damaged))
auc, ns = GeoTfmEval.calc(device, learner, test_files, labels, params.n_class)
auc

0.9002777777777778

In [10]:
from dlcliche.math import np_describe
np_describe(np.array(ns))

,count,mean,std,min,25%,50%,75%,max
0,90.0,0.999986,0.000022,0.999856,0.999988,0.999993,0.999995,0.999997


## Test case 2

- Good: jpg
- Others: jpg

In [11]:
test2_good = make_resized_files((ORG/'test/good').glob('*.png'), dest_folder='temp/test2/good',
                               suffix='.jpg', load_size=params.load_size)
test2_bend = make_resized_files((ORG/'test/bent_lead').glob('*.png'), dest_folder='temp/test2/bend',
                               suffix='.jpg', load_size=params.load_size)
test2_cut = make_resized_files((ORG/'test/cut_lead').glob('*.png'), dest_folder='temp/test2/cut',
                               suffix='.jpg', load_size=params.load_size)
test2_damaged = make_resized_files((ORG/'test/damaged_case').glob('*.png'), dest_folder='temp/test2/damaged',
                               suffix='.jpg', load_size=params.load_size)

test2_files = test2_good + test2_bend + test2_cut + test2_damaged
labels = np.array([0] * len(test2_good) + [1] * len(test2_bend) + [1] * len(test2_cut) + [1] * len(test2_damaged))
auc, ns = GeoTfmEval.calc(device, learner, test2_files, labels, params.n_class)
auc

0.8947222222222222

In [12]:
np_describe(np.array(ns))

,count,mean,std,min,25%,50%,75%,max
0,90.0,0.999986,0.000022,0.999854,0.999988,0.999993,0.999995,0.999997


## Test case 3

- Good: png
- Others: png

All test files are .png, though trained with .jpg files.

In [13]:
test3_good = make_resized_files((ORG/'test/good').glob('*.png'), dest_folder='temp/test3/good',
                               suffix='.png', load_size=params.load_size)
test3_bend = make_resized_files((ORG/'test/bent_lead').glob('*.png'), dest_folder='temp/test3/bend',
                               suffix='.png', load_size=params.load_size)
test3_cut = make_resized_files((ORG/'test/cut_lead').glob('*.png'), dest_folder='temp/test3/cut',
                               suffix='.png', load_size=params.load_size)
test3_damaged = make_resized_files((ORG/'test/damaged_case').glob('*.png'), dest_folder='temp/test3/damaged',
                               suffix='.png', load_size=params.load_size)
test3_files = test3_good + test3_bend + test3_cut + test3_damaged
labels = np.array([0] * len(test3_good) + [1] * len(test3_bend) + [1] * len(test3_cut) + [1] * len(test3_damaged))
auc, ns = GeoTfmEval.calc(device, learner, test3_files, labels, params.n_class)
auc

0.8733333333333334

In [14]:
np_describe(np.array(ns))

,count,mean,std,min,25%,50%,75%,max
0,90.0,0.999986,0.000022,0.999856,0.999988,0.999992,0.999994,0.999996
